In [7]:
import pandas as pd
import numpy as np

In [8]:
with open('dialog_acts.dat', 'r') as f:
    data = f.readlines()
    data = list(map(lambda x: [x[0:x.index(" ")], x[x.index(" ")+1:-1]], data))

In [9]:
df = pd.DataFrame(np.array(data), columns = ['label', 'text'])

In [10]:
df.head()

,label,text
0,inform,im looking for a moderately priced restaurant ...
1,inform,any part of town
2,inform,bistro food
3,confirm,is there a moderately priced restaurant that s...
4,affirm,yes


In [ ]:
from sklearn.model_selection import train_test_split

# Pre-Processing

In [11]:
df.head()

,label,text
0,inform,im looking for a moderately priced restaurant ...
1,inform,any part of town
2,inform,bistro food
3,confirm,is there a moderately priced restaurant that s...
4,affirm,yes


In [12]:
df['text'].head()

0    im looking for a moderately priced restaurant ...
1                                     any part of town
2                                          bistro food
3    is there a moderately priced restaurant that s...
4                                                  yes
Name: text, dtype: object

In [13]:
# if text contains "noise" or "tv_noise", remove
# if that makes the column empty, remove the row

for index, row in df.iterrows():
    print(row['c1'], row['c2'])

KeyError: ignored

#bot

In [5]:
#Your code should offer a prompt to enter a new utterance and classify this utterance, and repeat the prompt until the user exits.

def classify_utterance_1(ut):
  classified_label = "inform"
  return classified_label

def classify_utterance_2(ut):
  if 'bye' in ut:
    classified_label = 'bye'
  elif 'no' in ut:
    classified_label = 'negate'
  elif 'start over' in ut:
    classified_label = 'restart'
  else:
    classified_label = 'null'

  return classified_label

def classify_utterance_3(ut):
  #classified_label = ml_lassifier_model.predict(ut)[0]
  classified_label = "inform"
  return classified_label

def bot():
  finished = False
  print(f'Hi! To exit enter "bye123"')
  while not finished:
    utterance = input('>').lower()
    label = classify_utterance_3(utterance)
    print(utterance)  
    if 'bye123' in utterance:
      finished = True

In [6]:
bot()

Hi! To exit enter "bye123"
>you
you
>mine
mine
>hate you
hate you
>bye123
bye123


#ML model comparison

bag-of-word method means splitting sentance ito array of words  
its advised then to remove english stop words and puctuation  
then you have to vectorize them so that each word correspond to a specific number. (ML models deal better with number instead of word) using e.g. CountVectorizer  
then we have to "order" the workd 

In [15]:
df

,label,text
0,inform,im looking for a moderately priced restaurant ...
1,inform,any part of town
2,inform,bistro food
3,confirm,is there a moderately priced restaurant that s...
4,affirm,yes
...,...,...
25496,request,what is the price range
25497,thankyou,thank you good bye
25498,inform,im looking for african food
25499,request,whats the address and phone number


In [16]:
df['label_id'] = df['label'].factorize()[0]

In [18]:
df

,label,text,label_id
0,inform,im looking for a moderately priced restaurant ...,0
1,inform,any part of town,0
2,inform,bistro food,0
3,confirm,is there a moderately priced restaurant that s...,1
4,affirm,yes,2
...,...,...,...
25496,request,what is the price range,3
25497,thankyou,thank you good bye,4
25498,inform,im looking for african food,0
25499,request,whats the address and phone number,3


In [22]:
from sklearn.model_selection import train_test_split
text_train, text_test, label_train, label_test = train_test_split(df['text'], df['label_id'], test_size=0.15, random_state=10)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [24]:
tfidf = TfidfVectorizer(sublinear_tf=True, #scale the words frequency in logarithmic scale
                        min_df=5, #remove the words which has occurred in less than ‘min_df’ number of files
                        ngram_range=(1, 2), #dont know what role n-grams play in vectorisation
                        stop_words='english', #it removes stop words which are predefined in ‘english’.
                        lowercase=True #everything to lowercase
                        )

features = tfidf.fit_transform(text_train).toarray()
labels = label_train

In [27]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [35]:
models = [
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0, max_iter=400),
]

In [36]:
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))

In [37]:
from sklearn.model_selection import cross_val_score

In [38]:
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


In [39]:
mean_accuracy = cv_df.groupby('model_name').accuracy.mean()
std_accuracy = cv_df.groupby('model_name').accuracy.std()

acc = pd.concat([mean_accuracy, std_accuracy], axis= 1, 
          ignore_index=True)
acc.columns = ['Mean Accuracy', 'Standard deviation']
acc

,Mean Accuracy,Standard deviation
model_name,,
LinearSVC,0.890427,0.002468
LogisticRegression,0.879954,0.002417
MultinomialNB,0.857070,0.001920
RandomForestClassifier,0.682860,0.025147


In [40]:
#seems like Linear Support Vector Machine is the best or we can tweek the hyperparameters more. 
#I'm not very good at that stuff so any recomendations are welcome.

In [42]:
tfidf = TfidfVectorizer(sublinear_tf=True, #scale the words frequency in logarithmic scale
                        min_df=5, #remove the words which has occurred in less than ‘min_df’ number of files
                        ngram_range=(1, 2), #dont know what role n-grams play in vectorisation
                        stop_words='english', #it removes stop words which are predefined in ‘english’.
                        lowercase=True #everything to lowercase
                        )

features_test = tfidf.fit_transform(text_test).toarray()

In [43]:
model = LinearSVC()
model.fit(features, labels)
label_pred = model.predict(features_test)

ValueError: ignored

In [ ]:
print(metrics.classification_report(label_test, label_pred, 
                                    target_names= df['label'].unique()))

#exploration

In [59]:
df.describe()

,label,text
count,25501,25501
unique,15,5359
top,inform,thank you good bye
freq,10160,2565


In [72]:
df.groupby('label').describe().sort_values(('text','count'))

text                                      
          count unique                      top  freq
label                                                
reqmore       5      1                     more     5
restart      14      7               start over     7
deny         27     13                    wrong    14
ack          28     20                  okay um     7
repeat       33     13                   repeat     6
hello        93     60                    hello    24
confirm     172    115  is it moderately priced    11
bye         266     41                 good bye   137
negate      435    133                       no   264
affirm     1156    166                      yes   803
null       1612    249                    noise   408
reqalts    1747    437   is there anything else   302
thankyou   3259     81       thank you good bye  2565
request    6494    961             phone number   914
inform    10160   3062              i dont care   613